### Characterising pixel response of BFS-U3-200S6M Sensor 📸

We want to model the non-linear response per pixel to incoming photons (of various ${\lambda}$ and across intensities).

There are multiple sources of noise that vary with wavelength (e.g. flicker or 1/f noise) and incoming intensity (e.g. shot noise) - 
to simplify analysis we will just fit a curve to each of these pixel responses outlined by:
https://www1.cs.columbia.edu/CAVE/publications/pdfs/Grossberg_PAMI04.pdf 

Lets try a gamma curve (bunch of lit refs that follow):
${f(E) = \alpha + \beta E^\gamma}$

Useful when performing phase retrieval (can only retrieve as good as your model represents)
(probably over-kill but this is an interesting task regardless)

Using 635nm laser source 🔴

In [2]:
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt


### BFS Detector Info
CCD (Charge Coupled Device) sensors can be descirbed by a single value for readout noise because all charge is transported across the chip and read at a single corner of the array.

CMOS (Complementrary Metal-Oxide Semiconductor) sensors however have individual amplifier circuits per pixel, with each pixel/photosite read individually - so need per pixel read out noise to descibe these. **The BFS is a CMOS** detector

Shutters closed/0s exposure will give you readout noise

### Load and Process Raw frames
**NOTE**: running this once is fine. Will take quite some time (~7hr on toliman lab PC). Reading directly from Morgana2 drive.

Lets do average at each pixel and use the SEM as the error

In [4]:
exit()
fname_to_get = "Z:\gpir9156/toliman/detector" # Reading from Morgana because there is just so much data (300GB worth...)
                                            # Ran this on toliman laptop because faster
                                            # to access Morgana on my Mac: "/Volumes/Morgana2/gpir9156/toliman/detector" 

fname_to_save = "data/toliman/detector/"
print("File path exists?: {}".format(os.path.exists(fname_to_get)))

# start with the luminance files for each exposure (80us and 500us)
count = 0 # for my sanity
for name in os.listdir(fname_to_get):
    dir_name = os.path.join(fname_to_get, name)
    if name.endswith(".npy") and name.startswith("luminance"):
        print("Count: {}".format(count))
        print(name)
        count += 1
        data = np.load(dir_name)
        mean = np.mean(data, axis=0)
        stdev = np.std(data, axis=0)
        n = data.shape[0]

        SEM = stdev/np.sqrt(n)

        if not name.startswith("luminance"):
            print("Wrong file format")
            break

        if "80us" in name:
            _idx = name[10:].find("_") 
            fname = fname_to_save + "80us_detector/luminance_" + name[10:10+_idx] 
            np.save(file=fname + "_mean.npy", arr = mean )
            np.save(file=fname + "_SEM.npy", arr = SEM )

        elif "500us" in name:
            _idx = name[10:].find("_") 
            fname = fname_to_save + "500us_detector/luminance_" + name[10:10+_idx] 
            np.save(file=fname + "_mean.npy", arr = mean )
            np.save(file=fname + "_SEM.npy", arr = SEM )

        else:
            print("Un-identified exposure in {}".format(name))

# shutters
# for some reason one batch is corrupted - excluding
exclude = ["shutters_80us_0gain_img_stack_batch_2.npy"]
shutters_80us_means, shutters_500us_means = [],[] #okay to avg/std these out at the end since n is same for all
shutters_80us_SEMS, shutters_500us_SEMS = [],[]
for name in os.listdir(fname_to_get):
    if name.endswith(".npy") and name.startswith("shutters") and name not in exclude:
        dir_name = os.path.join(fname_to_get, name)
        print("Count: {}".format(count))
        count +=1
        print(name)
        data = np.load(dir_name, allow_pickle=False)

        mean = np.mean(data, axis=0)
        stdev = np.std(data, axis=0)
    
        if "80us" in name:
            shutters_80us_means.append(mean)
            shutters_80us_SEMS.append(stdev)
        elif "500us" in name:
            shutters_500us_means.append(mean)
            shutters_500us_SEMS.append(stdev)
        else:
            print("Un-identified exposure in {}".format(name))

print(np.asarray(shutters_500us_means).shape)
print(np.asarray(shutters_80us_means).shape)
mean_shutters_80us = np.mean(np.asarray(shutters_80us_means), axis=0)
mean_shutters_500us = np.mean(np.asarray(shutters_500us_means), axis=0)
SEM_shutters_80us = np.std(np.asarray(shutters_80us_SEMS) , axis=0)/np.sqrt(600) #200 frames per batch
SEM_shutters_500us = np.std(np.asarray(shutters_500us_SEMS), axis=0)/np.sqrt(600)

fname = fname_to_save + "80us_detector/shutters"
np.save(file=fname + "_mean.npy", arr = mean_shutters_80us )
np.save(file=fname + "_SEM.npy", arr = SEM_shutters_80us )

fname = fname_to_save + "500us_detector/shutters"
np.save(file=fname + "_mean.npy", arr = mean_shutters_500us )
np.save(file=fname + "_SEM.npy", arr = SEM_shutters_500us )


File path exists?: True
/Volumes/Morgana2/gpir9156/toliman/detector/luminance_4000_500us_0gain_img_stack_batch_0.npy


#### Display Reduced Data 👁️

Luminance within 1% 

Detector is (3648, 5472) so not sure best way to display this, start with first 1000?

Break up detector into areas and overlap curves

{}


Take intensity sweeps at two different exposures 80ms (spider data and Jewel too) and 500ms gain 0 always

100 frames each 

0 cd/m^2 is me putting on shutters

cd/m^2 range: 
cd/m^2 uncertainty: 

Camera left to warm up (running) for 10min

Bandpass filter of 530-640nm for characterisation

In [ ]:
# Fave optimiser to fit curve?


#### Matrix representation

In [ ]:
# For a sweep of input, uniform intensities - measure the difference between pixels (flat field)